In [122]:
import unittest
import importlib as i
import cosmology as c
import deprecated as d
import numpy as np

In [142]:
class TestCosmology(unittest.TestCase):
    
    def test_set_default(self):
        c.cosm_model = [1,2,5,3,6]
        c.H_0 = c.O_M = c.O_L = c.O_B = 69
        self.assertEqual(c.cosm_model, [1,2,5,3,6])
        self.assertEqual(c.O_M, 69)
        self.assertEqual(c.H_0, 69)
        
        c.set_default()
        self.assertEqual(c.cosm_model, [0.286, 0.714, 0.049, 70])
        self.assertEqual(c.O_M, 0.286)
        self.assertEqual(c.O_L, 0.714)
        self.assertEqual(c.O_B, 0.049)
        self.assertEqual(c.H_0, 70*c.kmsMpc_to_s)
    
    def test_update_cmodel(self):
        new_model = [0.284, 0.755, 0.010, 50]
        c.update_cmodel(new_model)
        self.assertEqual(c.cosm_model, [0.284, 0.755, 0.010, 50])
        self.assertEqual(c.H_0, 50*c.kmsMpc_to_s)
        self.assertEqual(c.O_M, 0.284)
        self.assertEqual(c.O_L, 0.755)
        self.assertEqual(c.O_B, 0.010)
        
        new_model.append(69)
        self.assertEqual(c.cosm_model, [0.284, 0.755, 0.010, 50])
        
        c.set_default()
    
    def test_E(self):
        self.assertEqual(c.E(0), (c.O_M + c.O_L)**0.5)
        self.assertEqual(c.E(1), (8.0*c.O_M + c.O_L)**0.5)
        self.assertEqual(c.E(0.5), (3.375*c.O_M + c.O_L)**0.5)
        self.assertEqual(c.E(55.19482), ((56.19482)**3*c.O_M + c.O_L)**0.5)
        
        ans = np.array([(c.O_M + c.O_L)**0.5, (8.0*c.O_M + c.O_L)**0.5, (3.375*c.O_M + c.O_L)**0.5, ((56.19482)**3*c.O_M + c.O_L)**0.5])
        self.assertEqual(c.E(np.array([0,1,0.5,55.19482])).tolist(), ans.tolist())
    
    def test_sort_by_first(self):
        l1, l2 = c.sort_by_first([4,3,5,-1,2], [5,69,8,72,-11])
        self.assertEqual(l1.tolist(), [-1,2,3,4,5])
        self.assertEqual(l2.tolist(), [72,-11,69,5,8])
        
        l1, l2 = c.sort_by_first([5,4,3,2], [-5,-6,-7,8])
        self.assertEqual(l1.tolist(), [2,3,4,5])
        self.assertEqual(l2.tolist(), [8,-7,-6,-5])
        
        #check if sort in place
        l1, l2 = c.sort_by_first([9,9,3,9,9], [5,-2,3,-4,99])
        self.assertEqual(l1.tolist(), [3,9,9,9,9])
        self.assertEqual(l2.tolist(), [3,5,-2,-4,99])
        
        #check if it works for numpy arrays as well
        l1, l2 = c.sort_by_first(np.array([4,3,5,-1,2]), np.array([5,69,8,72,-11]))
        self.assertEqual(l1.tolist(), [-1,2,3,4,5])
        self.assertEqual(l2.tolist(), [72,-11,69,5,8])
        
        l1, l2 = c.sort_by_first(np.array([5,4,3,2]), np.array([-5,-6,-7,8]))
        self.assertEqual(l1.tolist(), [2,3,4,5])
        self.assertEqual(l2.tolist(), [8,-7,-6,-5])
        
        l1, l2 = c.sort_by_first(np.array([9,9,3,9,9]), np.array([5,-2,3,-4,99]))
        self.assertEqual(l1.tolist(), [3,9,9,9,9])
        self.assertEqual(l2.tolist(), [3,5,-2,-4,99])
    
    def test_D_C(self):
        self.assertAlmostEqual(c.D_C(2.12384), d.D_C_old(2.12384), delta=1e-7)
        self.assertAlmostEqual(c.D_C(2.12384), 1.26580797970764, delta=1e-11) # value calcualted through integral-calculator
        self.assertAlmostEqual(c.D_C(55.123854), 2.870559850206997, delta=1e-11) # value calcualted through integral-calculator
        
        input = np.array([50,1,3,25,4])
        np.testing.assert_almost_equal(c.D_C(input), d.D_C_old(input), 7)
        np.testing.assert_almost_equal(c.D_C(input), np.array([2.846083449712865, 0.7779423776121797, 1.504993571457544, 2.636332485477817, 1.699640679205983]), 11)
    
    #next up: test D_L
    def test_D_L(self):
        pass

In [143]:
i.reload(c)
unittest.main(argv=[''], verbosity=2, exit=False)

test_D_C (__main__.TestCosmology) ... ok
test_E (__main__.TestCosmology) ... ok
test_set_default (__main__.TestCosmology) ... ok
test_sort_by_first (__main__.TestCosmology) ... ok
test_update_cmodel (__main__.TestCosmology) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.252s

OK
